In [ ]:
def get_halo_profile_property(output_path,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,min_edge,max_edge,Nbins,CENTER_AROUND='POTENTIAL_MINIMUM',p_id=0):
    g_minus_1 = (5.0/3.0) - 1.0
    XH = 0.76
    #yhelium = (1-XH)/(4*XH)


    def min_dis(median_position, position,box_size):
        pos_1=position-median_position
        pos_2=position-median_position+boxsize
        pos_3=position-median_position-boxsize
        new_position_options=numpy.array([pos_1,pos_2,pos_3])
        get_minimum_distance=numpy.argmin(numpy.abs(new_position_options))
        return new_position_options[get_minimum_distance]
    boxsize=get_box_size(output_path)
    particle_property='Coordinates'
    group_positions,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

    particle_property='Masses'
    group_mass,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

    particle_property='Potential'
    group_potential,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)
    
    particle_property='GFM_Metallicity'
    group_metallicity,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

    particle_property='Velocities'
    group_velocities,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

    particle_property='InternalEnergy'
    group_internal_energy,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

    particle_property='ElectronAbundance'
    group_electron_abundance,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

    if (CENTER_AROUND=='MOST_MASSIVE_BLACKHOLE'):

        particle_property='ParticleIDs'

        bh_IDs,output_redshift=get_particle_property_within_groups(output_path,particle_property,5,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

        particle_property='Coordinates'

        bh_positions,output_redshift=get_particle_property_within_groups(output_path,particle_property,5,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)
        particle_property='Masses'
        bh_masses,output_redshift=get_particle_property_within_groups(output_path,particle_property,5,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)
        print("Calculating density around BH with ID:",(bh_IDs[bh_masses==numpy.amax(bh_masses)])[0])
        center=(bh_positions[bh_IDs==p_id])[0]
    if (CENTER_AROUND=='POTENTIAL_MINIMUM'):
        center=(group_positions[group_potential==numpy.amin(group_potential)])[0]
    transposed_group_positions=numpy.transpose(group_positions)
    vectorized_min_dis = numpy.vectorize(min_dis)
    x_dis=vectorized_min_dis(center[0],transposed_group_positions[0],boxsize)
    y_dis=vectorized_min_dis(center[1],transposed_group_positions[1],boxsize)
    z_dis=vectorized_min_dis(center[2],transposed_group_positions[2],boxsize)
    log_distances=numpy.log10(numpy.sqrt(x_dis**2+y_dis**2+z_dis**2))
    log_distance_bins=numpy.linspace(min_edge,max_edge,Nbins)
    binning=correlate.RBinning(log_distance_bins)
    bin_edges=binning.edges
    bin_centers=binning.centers
    xvel = group_velocities[:,0]
    #print(xvel)
    yvel = group_velocities[:,1]
    #print(yvel)
    zvel = group_velocities[:,2]
    #print(zvel)
    vel_magnitude = numpy.sqrt((xvel**2)+(yvel**2)+(zvel**2))
    #print(vel_magnitude)
    #print(len(vel_magnitude))
    xcoord=group_positions[:,0]
    ycoord=group_positions[:,1]
    zcoord=group_positions[:,2]
    mass_distribution=[]
    metallicity_distribution=[]
    velocity_distribution=[]
    internal_energy_distribution=[]
    electron_abundance_distribution=[]
    distance=(numpy.sqrt(xcoord**2+ycoord**2+zcoord**2))
    distance_distribution=[]
    
    for i in range(0,len(bin_edges)-1):
        left=bin_edges[i]
        right=bin_edges[i+1]
        mask=(log_distances>left)&(log_distances<right)
        mass_inside_bin=numpy.sum(group_mass[mask])
        mass_distribution.append(mass_inside_bin)
        metallicity_inside_bin=numpy.average(group_metallicity[mask])
        metallicity_distribution.append(metallicity_inside_bin)
        velocity_inside_bin=numpy.average(vel_magnitude[mask])
        velocity_distribution.append(velocity_inside_bin)
        internal_energy_inside_bin=numpy.average(group_internal_energy[mask])
        internal_energy_distribution.append(internal_energy_inside_bin)
        electron_abundance_inside_bin=numpy.average(group_electron_abundance[mask])
        electron_abundance_distribution.append(electron_abundance_inside_bin)
        distance_inside_bin=numpy.average(distance[mask])
        #print(numpy.shape(distance[mask]))
        distance_distribution.append(numpy.shape(distance[mask])[0])

    mass_distribution=numpy.array(mass_distribution)
    metallicity_distribution=numpy.array(metallicity_distribution)
    velocity_distribution=numpy.array(velocity_distribution)
    #print(velocity_distribution)
    internal_energy_distribution=numpy.array(internal_energy_distribution)
    electron_abundance_distribution=numpy.array(electron_abundance_distribution)
    mass_density=mass_distribution*3/4/3.14/(10**bin_centers)**3
    mu=(4*ac.MP)/(1+3*XH+4*XH*electron_abundance_distribution)
    temperature = g_minus_1*(internal_energy_distribution/ac.KB)*(10**10)*mu
    distance_distribution=numpy.array(distance_distribution)

    return bin_centers,mass_distribution,mass_density,metallicity_distribution,velocity_distribution,distance_distribution,temperature



In [ ]:
def get_DM_property(output_path,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,min_edge,max_edge,Nbins,CENTER_AROUND='POTENTIAL_MINIMUM',p_id=0):
    def min_dis(median_position, position,box_size):
        pos_1=position-median_position
        pos_2=position-median_position+boxsize
        pos_3=position-median_position-boxsize
        new_position_options=numpy.array([pos_1,pos_2,pos_3])
        get_minimum_distance=numpy.argmin(numpy.abs(new_position_options))
        return new_position_options[get_minimum_distance]
    boxsize=get_box_size(output_path)
    particle_property='Coordinates'
    group_positions,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

    particle_property='Potential'
    group_potential,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

    '''
    particle_property='Coordinates'
    group_coordinates,output_redshift=get_particle_property_within_groups(output_path,particle_property,p_type,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)
'''
    if (CENTER_AROUND=='MOST_MASSIVE_BLACKHOLE'):

        particle_property='ParticleIDs'

        bh_IDs,output_redshift=get_particle_property_within_groups(output_path,particle_property,5,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)

        particle_property='Coordinates'

        bh_positions,output_redshift=get_particle_property_within_groups(output_path,particle_property,5,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)
        particle_property='Masses'
        bh_masses,output_redshift=get_particle_property_within_groups(output_path,particle_property,5,desired_redshift_of_selected_halo,index_of_selected_halo,group_type='groups',list_all=False)
        print("Calculating density around BH with ID:",(bh_IDs[bh_masses==numpy.amax(bh_masses)])[0])
        center=(bh_positions[bh_IDs==p_id])[0]
    if (CENTER_AROUND=='POTENTIAL_MINIMUM'):
        center=(group_positions[group_potential==numpy.amin(group_potential)])[0]
    transposed_group_positions=numpy.transpose(group_positions)
    vectorized_min_dis = numpy.vectorize(min_dis)
    x_dis=vectorized_min_dis(center[0],transposed_group_positions[0],boxsize)
    y_dis=vectorized_min_dis(center[1],transposed_group_positions[1],boxsize)
    z_dis=vectorized_min_dis(center[2],transposed_group_positions[2],boxsize)
    log_distances=numpy.log10(numpy.sqrt(x_dis**2+y_dis**2+z_dis**2))
    
    log_distance_bins=numpy.linspace(min_edge,max_edge,Nbins)
    binning=correlate.RBinning(log_distance_bins)
    bin_edges=binning.edges
    bin_centers=binning.centers
    xcoord=group_positions[:,0]
    ycoord=group_positions[:,1]
    zcoord=group_positions[:,2]
    distance=(numpy.sqrt(xcoord**2+ycoord**2+zcoord**2))
    distance_distribution=[]


    for i in range(0,len(bin_edges)-1):
        left=bin_edges[i]
        right=bin_edges[i+1]
        mask=(log_distances>left)&(log_distances<right)
        distance_inside_bin=numpy.average(distance[mask])
        #print(numpy.shape(distance[mask]))
        distance_distribution.append(numpy.shape(distance[mask])[0])
    distance_distribution=numpy.array(distance_distribution)

    return bin_centers,distance_distribution
